# Extracting and Cleaning Text from Telegram ChatExport

This notebook parses all html files in Telegram chat export , extracts message metadata and text, cleans the text for analysis, and saves outputs as:
- A CSV file containing all parsed messages and fields
- One cleaned TXT file per day (messages concatenated per date)

The goal is a clean, analysis-ready corpus while preserving essential metadata.

In [90]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from pathlib import Path
import re
import os

## Setup and Working Directory

Before running this notebook, you need to:

1. **Download a chat history from Telegram:**
   - Open Telegram and export the desired chat (Settings → Chat Settings → Export Chat History)
   - Select HTML format (default format)

2. **Save the export in a folder:**
   - Create a folder in the same location as this notebook
   - Name the folder with a relevant name (e.g., "Alice_Weidel" or "Group_Chat_2024")
   - Copy all HTML files from the Telegram export into this folder

3. **Enter the folder name when prompted:**
   - In the next cell, the program will ask you to enter the folder name
   - **Important:** Enter exactly the same name you gave the folder, as this name is used to locate the files and name the output files later in the program

In [105]:
# Iterate all files html files
folder_name = input()

folder = f'{folder_name}'


 Jorge Buxadé


## Load and Parse the Telegram Export

- open and read the html file using UTF-8 encoding.
- Parse the HTML with BeautifulSoup (bs).
- We will later extract each message block (`div.message.default.clearfix`) to a tidy table.

In [106]:
# Make list of html files in the chat export folder
html_file_names = [i for i in os.listdir(folder) if i.endswith('.html')]

def parse_into_bs(html_file):
    
    html_path = Path(rf'./{folder_name}/{html_file}')
    with html_path.open(encoding="utf-8") as f:
        html = f.read()
    
    soup = bs(html, "html.parser")

    return soup

# from html to soup elements
bs4_soup = [parse_into_bs(html_file) for html_file in html_file_names]

## Functions for Parsing Messages

This section:
- Extracts message date/time from the `"title"` attribute (e.g., `02.09.2025 13:45:12 UTC+0`)
- Reads the sender name (`div.from_name`)
- Gets text content (`div.text`), preserving line breaks
- Builds a pandas DataFrame with columns:
  - `message_id`, `date`, `time`, `utc`, `from_name`, `text`

In [107]:
def get_datetime_title(body):
    el = body.select_one("div.pull_right.date.details")
    title = el.get("title", "").strip() if el else ""
    date, time, utc = "", "", ""
    if title:
        parts = title.split(" ", 2)
        if len(parts) == 3:
            date_raw, time, utc = parts
            dparts = date_raw.split(".")
            if len(dparts) == 3 and dparts[0].isdigit() and dparts[1].isdigit():
                date = f"{int(dparts[0])}.{int(dparts[1])}.{dparts[2]}"
            else:
                date = date_raw
        else:
            date = title
    return date, time, utc


def get_from_name(body):
    el = body.select_one("div.from_name")
    return el.get_text(strip=True) if el else ""


def get_text(body):
    el = body.select_one("div.text")
    if not el:
        return ""
    for br in el.find_all("br"):
        br.replace_with("\n")
    return el.get_text(strip=True, separator="\n")


def parse_message_div(msg):
    body = msg.select_one("div.body")
    if body is None:
        return {}
    date, time, utc = get_datetime_title(body)
    return {
        "message_id": msg.get("id", ""),
        "date": date,
        "time": time,
        "utc": utc,
        "from_name": get_from_name(body),
        "text": get_text(body),
    }

def parse_all_messages_to_df(soup):
    msgs = soup.select("div.message.default.clearfix")
    rows = [parse_message_div(m) for m in msgs]
    rows = [r for r in rows if r]  # fjern tomme
    return pd.DataFrame(rows, columns=[
        "message_id",
        "date",
        "time",
        "utc",
        "from_name",
        "text",
    ])


# Build DataFrame - parse the html data into a dataframe

_df = [parse_all_messages_to_df(soup) for soup in bs4_soup]

_df = pd.concat(_df).reset_index(drop=True)

df = _df.sort_values('date', ascending=True)

## Build the DataFrame and Clean Text

- Construct the DataFrame from all parsed messages.
- Clean the raw text:
  - Replace newlines with spaces
  - Remove URLs
  - Collapse multiple spaces
  - Keep word tokens only
  - Lowercase
- Add:
  - `clean_text`: cleaned message text
  - `word_count`: number of words in `clean_text`

In [108]:
# Clean the text
def clean_text(text):
    _text = text.replace('\n', ' ') # remove new lines
    _text = re.sub(r'http.+\b', ' ',  _text) # remove links
    _text = re.sub(r'\s+', ' ', _text) # remove multiple white space
    _text = re.findall(r'\b\S+\b', _text) # remove all signs but letters - return list
    _text = ' '.join(_text) # join list to string 
    _text = _text.lower() # lower all letters
    return _text.strip() # strip white spaces

# Add new column with clean text
df['clean_text'] = df['text'].apply(clean_text)

# Add the number of words in the clean text
df['word_count'] = df['clean_text'].apply( lambda x : len(x.split())) 

# Filter data so only rows with word_count more than 10 are kept
df = df.query('word_count > 9')


In [109]:
df

,message_id,date,time,utc,from_name,text,clean_text,word_count
1815,message1850,1.1.2025,22:06:03,UTC+01:00,Jorge Buxadé 🇪🇸,"#Programa1Minuto\nTarjeta sanitaria única, igu...",programa1minuto tarjeta sanitaria única igual ...,38
1816,message1851,1.1.2025,22:07:11,UTC+01:00,,"Frente a su odio y su blasfemia, amor, belleza...",frente a su odio y su blasfemia amor belleza y...,19
321,message342,1.12.2020,22:23:11,UTC+01:00,Jorge Buxadé 🇪🇸,‼️Frente al Gobierno que atenta contra el orde...,frente al gobierno que atenta contra el orden ...,57
1925,message1960,1.2.2025,20:55:24,UTC+01:00,Jorge Buxadé 🇪🇸,🔴 Bajo investigación la INJERENCIA DE LOS BURÓ...,bajo investigación la injerencia de los burócr...,35
2010,message2045,1.3.2025,10:04:30,UTC+01:00,Jorge Buxadé 🇪🇸,Jornada ayer en el Pulgar con afectados por la...,jornada ayer en el pulgar con afectados por la...,72
...,...,...,...,...,...,...,...,...
2326,message2362,9.5.2025,13:22:33,UTC+01:00,Jorge Buxadé 🇪🇸,"🔴 MÁS EXPROPIACIONES DE OLIVOS 🔴\nEn España, e...",más expropiaciones de olivos en españa el nego...,51
2526,message2562,9.7.2025,19:53:53,UTC+01:00,Jorge Buxadé 🇪🇸,Quiero dar la enhorabuena a Irene Montero por ...,quiero dar la enhorabuena a irene montero por ...,288
2632,message2669,9.9.2025,09:07:04,UTC+01:00,Jorge Buxadé 🇪🇸,"Mismo día, tres teorías: que si el PP consider...",mismo día tres teorías que si el pp considera ...,99
2635,message2672,9.9.2025,13:02:28,UTC+01:00,Jorge Buxadé 🇪🇸,"🔴 El próximo lunes, 15 de septiembre, en Tener...",el próximo lunes 15 de septiembre en tenerife ...,26


## Save to CSV

Export data to a CSV file for later analysis
(e.g., in spreadsheets, Python, R, or Orange).


In [110]:
out_dir = Path("csv_file")
out_dir.mkdir(exist_ok=True)

path = os.path.join(f"./{out_dir}/{folder}_export.csv")
df.to_csv(path, index=False, encoding='utf-8')

## Export: Concatenate Cleaned Text by Day

- Convert `date` to a proper datetime (day-first).
- Group by day and concatenate all `clean_text` messages for that day.
- Save one TXT per date into `txt_files_grouped_by_day/`.

This is useful for daily-level analysis in tools like Voyant.

In [111]:
_df = df.copy()
_df['date'] = pd.to_datetime(_df['date'], dayfirst=True)
df_grouped = _df.groupby('date')['clean_text'].apply( lambda x : ' '.join(x)).reset_index() 
df_grouped = df_grouped.rename(columns={'clean_text': 'concatenated_text'})

out_dir = Path("txt_files_grouped_by_day")
out_dir.mkdir(exist_ok=True)


def make_txt_filename(date_str):
    date_str = str(date_str).strip().replace("-", "_")[0:10]
    if not date_str:
        date_str = "unknown_date"
    
    return f'{date_str}.txt'


written = 0
for index, row in df_grouped.iterrows():
    text = str(row['concatenated_text']).strip()
    if not text:
        continue
    _fname = make_txt_filename(row.get(("date"), ""))
    fname = folder + '_' + _fname
    (out_dir / fname).write_text(text, encoding="utf-8")
    written += 1

print(f"Wrote {written} txt files in {out_dir}")

Wrote 332 txt files in txt_files_grouped_by_day
